## LSTM

Heute möchten wir ein LSTM trainieren, um Äußerungen zu klassifizieren. Wir können dies für unterschiedlichste Anwendungsfälle nutzen, immer wenn ein Text zu einer Klasse zugeordnet werden muss. Dies kann ein Intent sein, aber auch eine Emotion oder Sentiment (positive, negative Stimmung).

Um ein LSTM zu trainieren setzen wir die Bibliotheken ```keras``` und ```tensorflow``` ein, installiere diese:


In [ ]:
pip install keras

In [ ]:
pip install scikit-learn

In [ ]:
pip install tensorflow-macos

In [ ]:
pip install tensorflow-metal

### Datensatz

Wir erstellen eine Liste an Sätzen und dazugehörige Labels. Such dir gerne einen Datensatz, der dich interessiert. https://huggingface.co/ oder https://www.kaggle.com/ sind gute Anlaufstellen, dort Datasets und mit ```sentiment``` oder  ```ìntent``` suchen. Zur Einbindung des Datensatzes kannst du ihn entweder herunterladen oder direkt über ```huggingface_hub``` (Anmeldung notwendig) einbinden.

In [ ]:
pip install huggingface_hub

In [ ]:
pip install datasets

In [ ]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("mteb/tweet_sentiment_extraction")
# the dataset is split into test and train, however, we want to make our own splits later on, thus we merge them into one single dataframe
df = pd.concat([pd.DataFrame(ds['test']), pd.DataFrame(ds['train'])])
df.head(10)

Bei realen Datensätzen ist es meist hilfreich explorativ den Datensatz zu untersuchen. Du kannst die nltk Bibliothek nutzen, um die Anzahl der Sätze und Wörter herausbekomen.

In [ ]:
from nltk.tokenize import word_tokenize

longest_text = max(df['text'], key=len)
vocabulary = []
for t in df['text']:
    vocabulary += word_tokenize(t, language='english')
vocabulary = list(set(vocabulary))

In [ ]:
print("Classes: ", set(df['label_text']))
print("Longest text: ", len(longest_text))
print("Vocabulary size: ", len(vocabulary))

### Modell

Ein guter Programmierstil ist die Hyperparameter als Konstanten anzulegen, um damit später etwas zu spielen:

In [ ]:
EMBEDDING_DIMENSION = 100
LSTM_UNITS = 64
NUM_CLASSES = len(set(df['label_text']))
# eigentlich sollte man die längeste Textsequenz nehmen. Bei Texten ungleicher Länge führt dies jedoch zu 
# sparse-Vektoren, was das Training erschwert.
TOKENIZATION_OUTPUT_SEQUENCE_LENGTH = 33
TEST_SIZE = 0.1
VALIDATION_SPLIT = 0.2
SEED = 35
BATCH_SIZE = 32
EPOCHS = 10

Wir müssen unsere Texte in Zahlenvektoren überführen. Hierfür ist ```TextVectorization``` hilfreich, das aus einem Text ein Vokabular aufbaut (```adapt()```) und anschließend den Text vektorisiert. Wie üblich bei Klassifikationsproblemen kodieren wir die Labels als Hot-Encoded Vektoren (```LabelEncoder```). Anschlißend nutzen wir ```train_test_split()```, um unseren Datensatz in einen Test- und Trainingsdaten zu splitten.

In [ ]:
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.layers import TextVectorization

vectorize_layer = TextVectorization(output_sequence_length=TOKENIZATION_OUTPUT_SEQUENCE_LENGTH)
vectorize_layer.adapt(df['text'])
text_encoded = vectorize_layer(df['text']).numpy()

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['label_text'])
labels_categorical = tf.keras.utils.to_categorical(labels, NUM_CLASSES)

X_train, X_test, y_train, y_test = train_test_split(text_encoded, labels_categorical, test_size=TEST_SIZE, random_state=SEED)

Nun kompilieren wir unser LSTM Modell mit ```keras``` und trainieren das Netz, hilfreiche Layer sind ```Èmbedding```, ```LSTM```, ```Dense```, die sequentiell hintereinander ausgeführt werden. Wir trainieren unsere Embeddings hier direkt mit in den Modell-Layern:
- Embedding: https://keras.io/api/layers/core_layers/embedding/
- LSTM: https://keras.io/api/layers/recurrent_layers/lstm/ 
- Dense: https://keras.io/api/layers/core_layers/dense/

In [ ]:
from keras.layers import Embedding, LSTM, Dense
from keras.models import Sequential

model = Sequential()
model.add(Embedding(len(vocabulary), EMBEDDING_DIMENSION, input_length = X_train.shape[1]))
model.add(LSTM(LSTM_UNITS))
model.add(Dense(NUM_CLASSES, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=VALIDATION_SPLIT)

Wie sehen die Loss-Curves aus?

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(range(1, EPOCHS + 1), history.history['loss'], label='Training Loss', marker='o')
plt.plot(range(1, EPOCHS + 1), history.history['val_loss'], label='Validation Loss', marker='x')
plt.title('Loss Curve')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

Nach dem Training können wir einzelne Sätze ausprobieren. Gebe dem Modell einen Text und lasse ihn klassifizieren:

In [ ]:
import numpy as np

twt_s = "I love it"
twt = vectorize_layer(twt_s).numpy()
result = model.predict(twt.reshape(1, X_train.shape[1]), batch_size=BATCH_SIZE, verbose="auto")

class_names = ['negative', 'neutral', 'positive']
predicted_class_number = np.argmax(result)


print(f"'{twt_s}' is {class_names[predicted_class_number]} ({predicted_class_number})")

Nun wird es spannend :). Wie schneidet unser Modell mit dem Testdatensatz ab?

In [ ]:
score,acc = model.evaluate(X_test, y_test, verbose = 2, batch_size = BATCH_SIZE)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))